In [1]:
import pandas as pd

In [2]:
# Load backtest results
file_path = '..\\data\\backtest_results.csv'  # Update with your actual file path
data = pd.read_csv(file_path)

# Ensure datetime format if there's a timestamp column
if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Num', inplace=True)

data.drop(columns=['Swap', 'Symbol', 'Order', 'Unnamed: 0'], inplace=True)

# Inspect the data
data.head()

,Date,Type,Direction,Volume,Price,Commission,Profit,Balance,Comment
Num,,,,,,,,,
1,2023-01-03 01:00:39,buy,in,0.96,1 828.67,- 2.88,0.00,99 997.12,NaN
2,2023-01-03 13:00:02,sell,out,0.96,1 834.42,0.00,552.00,100 549.12,NaN
3,2023-01-03 15:17:31,buy,in,1.31,1 838.97,- 3.93,0.00,100 545.19,NaN
4,2023-01-03 17:28:39,sell,out,1.31,1 832.77,0.00,- 812.20,99 732.99,sl 1832.77
5,2023-01-03 17:28:44,buy,in,26.64,1 833.00,- 79.92,0.00,99 653.07,NaN


In [3]:
# Clean numeric columns (remove spaces and convert to float)
numeric_cols = ['Price', 'Commission', 'Profit', 'Balance']
for col in numeric_cols:
    data[col] = data[col].str.replace(' ', '').astype(float)
    
# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'])
data.head()

,Date,Type,Direction,Volume,Price,Commission,Profit,Balance,Comment
Num,,,,,,,,,
1,2023-01-03 01:00:39,buy,in,0.96,1828.67,-2.88,0.0,99997.12,NaN
2,2023-01-03 13:00:02,sell,out,0.96,1834.42,0.00,552.0,100549.12,NaN
3,2023-01-03 15:17:31,buy,in,1.31,1838.97,-3.93,0.0,100545.19,NaN
4,2023-01-03 17:28:39,sell,out,1.31,1832.77,0.00,-812.2,99732.99,sl 1832.77
5,2023-01-03 17:28:44,buy,in,26.64,1833.00,-79.92,0.0,99653.07,NaN


Pair Entry and Exit Trades

In [4]:
trades = []

# Iterate through pairs of rows (entry 'in' and exit 'out')
for i in range(0, len(data)-1, 2):
    entry = data.iloc[i]
    exit = data.iloc[i+1]
    
    if entry['Direction'] == 'in' and exit['Direction'] == 'out':
        trade = {
            'EntryDate': entry['Date'],
            'ExitDate': exit['Date'],
            'Type': entry['Type'],
            'Volume': entry['Volume'],
            'EntryPrice': entry['Price'],
            'ExitPrice': exit['Price'],
            'Profit': exit['Profit'],
            'StopLoss': 'sl' in str(exit['Comment']).lower(),
            'Duration': (exit['Date'] - entry['Date']).total_seconds() / 60  # in minutes
        }
        trades.append(trade)

trades_df = pd.DataFrame(trades)

Feature Engineering

In [5]:
# Extract time-based features
trades_df['EntryHour'] = trades_df['EntryDate'].dt.hour
trades_df['EntryDay'] = trades_df['EntryDate'].dt.day_name()

# Calculate return percentage
trades_df['ReturnPct'] = trades_df['Profit'] / (trades_df['Volume'] * trades_df['EntryPrice'])

trades_df.head()
trades_df.to_csv('..\\data\\trades.csv')